# Approve DTA Version

Promotes the current draft to a DTA Major version (triggered by workflow approval).

**Operation:** `APPROVE_DTA`

**What it does:**
- Closes the current draft (sets `is_current=FALSE`, `effective_end_ts=NOW()`)
- Creates new DTA Major version with `is_dta_major=TRUE`
- Version format: `{base}-{dta_id}-v1.0` (or v2.0, v3.0, etc.)
- Updates `md_version_registry` (draft -> SUPERSEDED, major -> ACTIVE)

**Parameters:**
- `dta_id` - DTA identifier (required)
- `library_type` - Library type (e.g., transfer_variables)

**Trigger:** Called when DTA workflow is fully approved.


In [ ]:
# Cell 1: Imports
import json
from clinical_data_standards_framework.versioning import (
    approve_dta_version,
    get_dta_version_history
)

print("="*80)
print("APPROVE DTA VERSION")
print("="*80)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
versioning_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="versioning"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
gold_schema = globals_dict['gold_schema']

# Versioning configuration from YAML
library_tables = versioning_dict.get('library_tables', [])
registry_table_name = versioning_dict.get('registry_table', 'md_version_registry')

print(f"Catalog: {catalog}")
print(f"Gold Schema: {gold_schema}")
print(f"Versioning Config: {len(library_tables)} library table(s)")
print(f"Created by: {created_by_principal}")


In [ ]:
# Cell 3: Get Widget Parameters (job-level parameters only)
dbutils.widgets.text("dta_id", "", "DTA ID")
dbutils.widgets.text("library_type", "", "Library Type (defaults from config)")

dta_id = dbutils.widgets.get("dta_id")
library_type_param = dbutils.widgets.get("library_type")

# Default to first library type from config if not specified
if library_type_param:
    library_type = library_type_param
elif library_tables:
    library_type = library_tables[0].get('library_type', 'transfer_variables')
else:
    library_type = 'transfer_variables'

print(f"\nDTA ID: {dta_id}")
print(f"Library Type: {library_type} (from {'param' if library_type_param else 'config'})")

# Validate
if not dta_id:
    raise ValueError("dta_id parameter is required")


In [ ]:
# Cell 4: Configure Table Names (from config)
# Build library table map from config
library_table_map = {}
for lib_config in library_tables:
    lib_type = lib_config.get('library_type')
    lib_name = lib_config.get('name')
    lib_schema = lib_config.get('schema', gold_schema)
    if lib_type and lib_name:
        library_table_map[lib_type] = f"{catalog}.{lib_schema}.{lib_name}"

if library_type not in library_table_map:
    available = ", ".join(library_table_map.keys())
    raise ValueError(f"Unknown library_type: {library_type}. Available: {available}")

library_table = library_table_map[library_type]
registry_table = f"{catalog}.{gold_schema}.{registry_table_name}"

print(f"\nLibrary Table: {library_table}")
print(f"Registry Table: {registry_table}")


In [ ]:
# Cell 5: Execute APPROVE_DTA
print(f"\n{'='*80}")
print(f"Approving DTA {dta_id} - promoting to DTA Major version...")
print(f"{'='*80}")

result_version = approve_dta_version(
    spark=spark,
    library_table=library_table,
    registry_table=registry_table,
    dta_id=dta_id,
    library_type=library_type,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id
)

print(f"\nDTA approved successfully!")
print(f"New DTA Major version: {result_version}")


In [ ]:
# Cell 6: Set Task Values for Downstream
dbutils.jobs.taskValues.set(key="result_version", value=result_version)
dbutils.jobs.taskValues.set(key="dta_id", value=dta_id)
dbutils.jobs.taskValues.set(key="action", value="APPROVE_DTA")

print(f"\nTask values set:")
print(f"  result_version: {result_version}")
print(f"  dta_id: {dta_id}")
print(f"  action: APPROVE_DTA")


In [ ]:
# Cell 7: Show Version History
print(f"\n{'='*80}")
print(f"Version History for DTA: {dta_id}")
print(f"{'='*80}")

history_df = get_dta_version_history(spark, registry_table, dta_id)
history_df.select(
    "version_tag", 
    "version_type", 
    "status", 
    "record_count",
    "parent_version",
    "created_ts"
).show(truncate=False)
